<a href="https://colab.research.google.com/github/MariaChuquitarco/014MBID/blob/main/TFM_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Proceso KDD


##Obtención de la base de datos

In [13]:
import pandas as pd
base_personas= pd.read_csv("/content/BDD_ENDI_R1_f1_personas_Trabajada.csv", delimiter=';')
base_hogar = pd.read_csv("/content/BDD_ENDI_R1_f1_hogar.csv", delimiter=';')
base_lactancia = pd.read_csv("/content/BDD_ENDI_R1_f2_lactancia.csv", delimiter=';')
base_mef = pd.read_csv("/content/BDD_ENDI_R1_f2_mef.csv", delimiter=';')
base_salud_ninez = pd.read_csv("/content/BDD_ENDI_R1_f2_salud_ninez.csv", delimiter=';')

<ipython-input-13-cab56364ea85>:2: DtypeWarning: Columns (17) have mixed types. Specify dtype option on import or set low_memory=False.
  base_personas= pd.read_csv("/content/BDD_ENDI_R1_f1_personas_Trabajada.csv", delimiter=';')
<ipython-input-13-cab56364ea85>:3: DtypeWarning: Columns (60,61,62,63,64,65) have mixed types. Specify dtype option on import or set low_memory=False.
  base_hogar = pd.read_csv("/content/BDD_ENDI_R1_f1_hogar.csv", delimiter=';')
<ipython-input-13-cab56364ea85>:5: DtypeWarning: Columns (71,72) have mixed types. Specify dtype option on import or set low_memory=False.
  base_mef = pd.read_csv("/content/BDD_ENDI_R1_f2_mef.csv", delimiter=';')
<ipython-input-13-cab56364ea85>:6: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  base_salud_ninez = pd.read_csv("/content/BDD_ENDI_R1_f2_salud_ninez.csv", delimiter=';')


A continunación el primer filtro teneindo en cuenta el criterio de  clasificación proporcionada por el Fondo de las Naciones Unidas para la Infancia (UNICEF)

In [19]:
columnas_personas= ['id_viv','id_hogar','id_per','edaddias','f1_s1_10','f1_s1_11','f1_s1_12','f1_s5_4_1','f1_s5_4_2','f1_s5_4_3',
                    'f1_s5_5_1','f1_s5_5_2','f1_s5_5_3','f1_s5_6_1','f1_s5_6_2','f1_s5_6_3','f1_s6_3','dcronica']
columnas_hogar=['id_viv','id_hogar','f1_s3_3','f1_s3_5','f1_s3_7','f1_s3_9','f1_s3_11','f1_s3_12','f1_s3_20','f1_s3_28','f1_s4_1_6',
                'f1_s7_1','f1_s7_4','f1_s7_23']
columnas_lactancia=['id_viv','id_hogar','id_per','id_mef_per','f2_s3_302','f2_s3_304','f2_s3_307_2','f2_s3_310_a_1','f2_s3_310_b_2',
                     'f2_s3_310_c_2','f2_s3_310_d_1','f2_s3_310_e_1','f2_s3_310_f_1','f2_s3_311','f2_s3_312_a','f2_s3_312_b','f2_s3_312_c',
                     'f2_s3_312_d','f2_s3_312_e','f2_s3_312_f','f2_s3_312_g','f2_s3_312_h','f2_s3_312_i','f2_s3_312_j','f2_s3_312_k',
                      'f2_s3_312_l','f2_s3_312_m','f2_s3_312_n','f2_s3_312_o','f2_s3_312_p','f2_s3_312_q','f2_s3_312_r','f2_s3_312_s',
                      'f2_s3_312_t','f2_s3_312_u','f2_s3_313','f2_s3_314','f2_s3_315','f2_s3_316','f2_s3_318','f2_s3_320','f2_s3_321','f2_s3_323']
columnas_mef=['id_viv','id_hogar','f2_s1_101','f2_s2_227_3','f2_s2_228_1','f2_s2_233_1','f2_s2_233_2','f2_s2_233_4']
columnas_salud_ninez=['id_viv','id_hogar','id_per','area','ord_hijo','f2_s4a_402','f2_s4b_404','f2_s4b_406','f2_s4b_409_a','f2_s4b_410_a','f2_s4b_411_a',
                      'f2_s4b_414','f2_s4b_415','f2_s4b_416','f2_s4b_417','f2_s4b_418_a','f2_s4b_419_a','f2_s4b_420_a','f2_s4b_421_a','f2_s4b_422_a',
                      'f2_s4b_423_a','f2_s4b_424_a','f2_s4b_425_a','f2_s4b_426_a','f2_s4b_427','f2_s4b_428a_1','f2_s4b_428a_2','f2_s4b_428a_3','f2_s4b_428a_4',
                      'f2_s4b_428a_5','f2_s4b_428b_6','f2_s4b_428a_7','f2_s4b_428a_8','f2_s4b_428a_9','f2_s4b_428a_10','f2_s4c_431','f2_s4d_436','f2_s4e_448_a',
                      'f2_s4e_448_b','f2_s4e_448_c']

personas_filtrado = base_personas[columnas_personas]
hogar_filtrado = base_hogar[columnas_hogar]
lactancia_filtrado = base_lactancia[columnas_lactancia]
mef_filtrado = base_mef[columnas_mef]
salud_ninez_filtrado = base_salud_ninez[columnas_salud_ninez]


Para una mejor comprención se cambiara de nombre a las variables dandole nombres estrategicos

In [51]:
# Asignar nuevos nombres a todas las columnas
personas_filtrado.columns = ['id_viv','id_hogar','id_per','edaddias','etnia','padre_hogar','madre_hogar','peso_1','peso_2','peso_3',
                    'long_1','long_2','long_3','talla_1','talla_2','talla_3','hemoglobina','dcronica']
hogar_filtrado.columns=['id_viv','id_hogar1','material_techo','material_paredes','material_piso','tipo_agua','servicio_higenico','dispone_luz_red_publica','cocina_mediante','existe_agua','quedo_sin_alimentos',
                'agua_proviene','ultimas_2_semanas_cantidades_necesarias','agua_que_beben']

lactancia_filtrado.columns=['id_viv','id_hogar2','id_per2','id_mef_per','dio_ceno_al_nacer','tiempo_despues_empezo_a_lactar','dias_solo_pecho','tomo_agua_pura_ayer','ayer_cuantos_tarros_de_leche',
                     'ayer_duantas_fundas_de_leche','ayer_tomo_jugos','ayer_comio_sopa','ayer_tomo_otro_liquido','ayer_comio_alimento_solido','ayer_comio_colada_espesa','ayer_comio_pure','ayer_comio_zapallo',
                     'ayer_comio_papa','ayer_comio_cualquier_vegetal','ayer_comio_maduro','ayer_comio_cualquier_fruta','ayer_comio_higado','ayer_comio_cualquier_carne','ayer_comio_huevos','ayer_comio_pescado',
                      'ayer_comio_cualquier semilla','ayer_comio_lacteo','ayer_comio_alimentos_preparados_manteca_cerdo','ayer_comio_dulce','ayer_comio_alimentos_preparados_condimentos','f2_s3_312_qayer_comio_alimentos_aceite_vegetal','ayer_comio_alimentos_preparados_aceite_vegetal','ayer_comio_alimentos_sal',
                      'ayer_comio_ala¡imentos_dulce','otros_alimentos','cuantas_veces_comio_solido','tomo_liquido_en_biceron','cuantos_biberones','liquido_que_toma','recibio_dos_tomas_de_leche','respetan_permiso:lactancia','respetan_licencia_maternidad','permiten_sala_lactancia']
mef_filtrado.columns=['id_viv','id_hogar3','años cumplidos','hijos_vivos_que_murieron','cuantos_hijos_fallecieron_5meses','total_hijos_viven_casa','total_hijos_fuera_casa','suma_hijos_vivos']

salud_ninez_filtrado.columns=['id_viv','id_hogar4','id_per4','area','ord_hijo','esta_vivo','tuvo_control_penatral','controles_antes_parto','consumo_acido_folico_durante_embarazo','consumo_hierro_durante_embarazo','consumo micronutrirntes_durante_embarazo',
                      'toma_presión_durante_embarazo','examen_VIH_antes_20semanas','examen_VIH_a_partir_20semanas','examen_VIH_durante_embarazo','examen_anemia_antes_20semanas','examen_anemia_a_oartir_20semanas','examen_orina__antes_20semanas','examen_orina_a_partir_20semanas','examen_TORCHs_antes_20semanas',
                      'examen_TORCHs_a_partir_20semanas','vacuna_tetanos_y_difteria','vacuna_influencia','ecos_obstetricos','recibio_charla_durante_embarazo','recibio_charla_sobre_lactancia','recibio_charla_consumo_micronutrientes','recibio_charla_alarma_embarazo','recibio_charla_preparacion_alimentos',
                      'recibio_charla_lavado_de_manos','cuantos_nacidos_vivos','recibio_consejeria_apego_lactancia','recibio_charla_alimentacion_saludable','recibio_charla_planificacion_transporte','recibio_charla_agua_segura','tipo_parto','primera_hora_leche_materna','primer_control_postarto_dias',
                      'primer_control_postarto_semanas','primer_control_postarto_meses']

Se unen las bases

In [60]:
# Unir personas y hogar
personas_hogar = personas_filtrado.merge(hogar_filtrado, on='id_viv', how='inner')

# Unir el resultado anterior con lactancia
personas_hogar_lactancia = personas_hogar.merge(lactancia_filtrado, on='id_viv', how='inner')

# Unir el resultado con mef
personas_hogar_lactancia_mef = personas_hogar_lactancia.merge(mef_filtrado, on='id_viv', how='inner')

# Finalmente, unir con salud_ninez
base_general = personas_hogar_lactancia_mef.merge(salud_ninez_filtrado, on='id_viv', how='inner')


Filtrar solo a los niños menore o igual a 5 años

In [61]:
# Definir el rango de días para niños de 5 años o menos
base_general['edaddias'] = pd.to_numeric(base_general['edaddias'], errors='coerce')
dias_max = 1825  # 5 años
# Filtrar el DataFrame para incluir solo a los niños de 5 años o menos
base_general = base_general[base_general['edaddias'] <= dias_max]

In [63]:
# Obtener la cantidad de columnas y filas
cantidad_columnas = base_general.shape[1]  # Número de columnas
cantidad_registros = base_general.shape[0]  # Número de filas

# Imprimir la cantidad de columnas y registros
print(f'Cantidad de columnas: {cantidad_columnas}')
print(f'Cantidad de registros: {cantidad_registros}')


Cantidad de columnas: 119
Cantidad de registros: 10154


### Exploratorio de datos

###Creación de los conjunto de validación

##Selección, prepocesamiento y transformación

###Tratamiento de valores nulos

###Tratamiento de valores atipicos

###Oversampling

###Tratamiento de variables

####Analisis de Componentes Especiales (ACP)

##Mineria de datos

###Regresión Logistica

###Random Forest

##Interpretación y Evalución

###Matriz de confusión

###Curva ROC

###Validación Cruzada